In [1]:
import psycopg2
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

In [2]:
# Define a database name (we're using a dataset on births, so we'll call it birth_db)
# Set your postgres username
dbname = 'map_the_vote'
username = 'codyschank' # change this to your username

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

postgres://codyschank@localhost/map_the_vote
True


In [3]:
voter_file = pd.read_csv('/Users/codyschank/Dropbox/Insight/voter_file_tx_25.csv',
                         dtype={"Voter File VANID": str, "Sex": str, "Age": str, "DateReg": str,
                                "StreetPrefix": str, "StreetName": str, "StreetType": str,
                                "AptType": str, "AptNo": str, "City": str, "State": str,
                                "Zip5": str}, keep_default_na=False)

/Users/codyschank/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
voter_file.head()

,Voter File VANID,Sex,Age,DateReg,StreetNo,StreetNoHalf,StreetPrefix,StreetName,StreetType,StreetSuffix,AptType,AptNo,City,State,Zip5,Zip4
0,1963191,F,38,10/14/2005,200,,,Timberline,Rd,,,,Wimberley,TX,78676,2319
1,2014843,F,66,9/15/2005,1224,,,River Mountain,Rd,,,,Wimberley,TX,78676,6382
2,2014913,F,77,3/1/1977,374,,,Fm 217,,,,,Valley Mills,TX,76689,3153
3,2014937,M,99,3/31/1977,101,,,County Road 1500,,,,,Morgan,TX,76671,3139
4,2014939,F,92,3/31/1977,1212,,,Mary,St,,,,Clifton,TX,76634,1328


In [5]:
voter_file.columns = map(str.lower, voter_file.columns)
voter_file.columns = voter_file.columns.str.replace('\s+', '_') 

In [7]:
# some preprocessing steps that will help later
voter_file["street_address"] = voter_file["streetno"].map(str) + ' ' +  voter_file["streetname"] + ' ' +  voter_file["streettype"] 
voter_file["residential"] = "y"
voter_file["street_address"] = voter_file["street_address"].str.upper()

In [9]:
# add multi-unit field

voter_file.loc[(voter_file['apttype'].str.len() > 0),'multi_unit'] = "y"
voter_file.loc[(voter_file['apttype'].str.len() == 0),'multi_unit'] = "n"
# voter_file['street_address'].head()
voter_file.head()

,voter_file_vanid,sex,age,datereg,streetno,streetnohalf,streetprefix,streetname,streettype,streetsuffix,apttype,aptno,city,state,zip5,zip4_,street_address,residential,multi_unit
0,1963191,F,38,10/14/2005,200,,,Timberline,Rd,,,,Wimberley,TX,78676,2319,200 TIMBERLINE RD,y,n
1,2014843,F,66,9/15/2005,1224,,,River Mountain,Rd,,,,Wimberley,TX,78676,6382,1224 RIVER MOUNTAIN RD,y,n
2,2014913,F,77,3/1/1977,374,,,Fm 217,,,,,Valley Mills,TX,76689,3153,374 FM 217,y,n
3,2014937,M,99,3/31/1977,101,,,County Road 1500,,,,,Morgan,TX,76671,3139,101 COUNTY ROAD 1500,y,n
4,2014939,F,92,3/31/1977,1212,,,Mary,St,,,,Clifton,TX,76634,1328,1212 MARY ST,y,n


In [8]:
table_name = "voter_file_tx25_all"
#write first 1000 rows to database
voter_file.head(1000).to_sql(table_name, engine, if_exists='replace')


In [9]:
chunk_size = 1000
for i in range(1000, voter_file.shape[0]+chunk_size, chunk_size):
    print(i)
    voter_file[i:(i+chunk_size)].to_sql(table_name, engine, if_exists='append')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [10]:
con = None
con = psycopg2.connect(database = dbname, user = username)

# query:
sql_query = """
SELECT * FROM voter_file_tx25_all;
"""
voter_file_from_sql = pd.read_sql_query(sql_query,con)
voter_file_from_sql.head()

,index,voter_file_vanid,sex,age,datereg,streetno,streetnohalf,streetprefix,streetname,streettype,streetsuffix,apttype,aptno,city,state,zip5,zip4_,street_address,residential
0,0,1963191,F,38,10/14/2005,200,,,Timberline,Rd,,,,Wimberley,TX,78676,2319,200 TIMBERLINE RD,y
1,1,2014843,F,66,9/15/2005,1224,,,River Mountain,Rd,,,,Wimberley,TX,78676,6382,1224 RIVER MOUNTAIN RD,y
2,2,2014913,F,77,3/1/1977,374,,,Fm 217,,,,,Valley Mills,TX,76689,3153,374 FM 217,y
3,3,2014937,M,99,3/31/1977,101,,,County Road 1500,,,,,Morgan,TX,76671,3139,101 COUNTY ROAD 1500,y
4,4,2014939,F,92,3/31/1977,1212,,,Mary,St,,,,Clifton,TX,76634,1328,1212 MARY ST,y
